In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lire les données
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Supprimer les lignes avec une cible manquante, séparer la cible des prédicteurs
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Pour simplifier, on utilise uniquement les prédicteurs numériques
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Séparer les données en ensemble d'entraînement et de validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [10]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


## Étape 1 : Enquête préliminaire
Le code que vous devez exécuter permet d'obtenir un aperçu de la forme des données d'entraînement et du nombre de valeurs manquantes dans chaque colonne. Examinons ce que fait chaque partie du code et comment en tirer les bonnes réponses :

#### 1. Affichage de la forme des données d'entraînement

In [14]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

(1168, 36)


#### 2. Calcul des valeurs manquantes par colonne

In [17]:
# Nombre de valeurs manquantes par colonne des données d'entraînement
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [21]:
num_rows = X_train.shape[0]  # Nombre de lignes
num_rows

1168

In [25]:
# Nombre de colonnes avec des valeurs manquantes
num_cols_with_missing = missing_val_count_by_column[missing_val_count_by_column > 0].shape[0]  
num_cols_with_missing

3

In [27]:
# Nombre total de valeurs manquantes
tot_missing = missing_val_count_by_column.sum()  
tot_missing

276

### Analyse
Dans ce cas, les données présentent relativement peu de valeurs manquantes (moins de 20 % des entrées dans la colonne ayant le plus de valeurs manquantes). Cela signifie que :

#### 1. Supprimer les colonnes entraînerait une perte importante d'informations :
- Même si certaines colonnes contiennent des valeurs manquantes, la plupart de leurs entrées sont valides.
- Supprimer ces colonnes revient à négliger des données qui pourraient être cruciales pour améliorer les prédictions.

#### 2. Imputer les valeurs manquantes permet de conserver les données existantes :
- En remplaçant les valeurs manquantes par des estimations (par exemple, la médiane), nous préservons les informations des colonnes sans introduire de biais significatif.
- Cela est particulièrement utile lorsque les données manquantes sont relativement peu nombreuses.


### Fonction score_dataset

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Fonction pour évaluer différentes approches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)


## Etape 2 : Suppression des colonnes avec des valeurs manquantes

In [35]:
# Supprimez les colonnes avec des valeurs manquantes
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

# Évaluer la performance
print("MAE (Suppression des colonnes):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Suppression des colonnes):
17837.82570776256


### Etape 3. Imputation 

#### Imputation avec la moyenne

In [44]:
from sklearn.impute import SimpleImputer

# Initialisation de l'imputer avec la stratégie "mean"
imputer = SimpleImputer(strategy='mean')

# Appliquer l'imputation sur les données d'entraînement et de validation
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# Restaurer les noms de colonnes après l'imputation
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

# Calculer et afficher le MAE
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


### Analyse des résultats (Part B)

Les résultats montrent que, contrairement aux attentes, **l'approche consistant à supprimer les colonnes a légèrement mieux fonctionné que l'imputation avec la moyenne**. Cela peut être surprenant, mais voici quelques explications potentielles :

---

### Raisons possibles pour lesquelles la suppression des colonnes a mieux fonctionné :
1. **Quantité limitée de données manquantes :** 
   - Lorsque seules quelques valeurs sont manquantes, leur suppression peut ne pas affecter significativement les performances du modèle.
   - Les colonnes avec des valeurs manquantes peuvent contenir moins d'informations utiles pour le modèle.

2. **Imputation inadéquate avec la moyenne :**
   - Dans certains cas, remplacer les valeurs manquantes par la moyenne peut introduire un bruit artificiel dans les données.
   - Par exemple, pour la colonne `GarageYrBlt` (année de construction du garage), une valeur manquante pourrait indiquer qu’il n’y a pas de garage. Remplacer cela par la moyenne (une année construite) ne reflète pas la réalité.

3. **Distribution des données :**
   - Si les valeurs manquantes sont corrélées à certaines propriétés importantes du dataset, remplacer par la moyenne peut perturber les relations entre les colonnes.

4. **Influence du bruit dans les données :**
   - Le bruit statistique peut parfois jouer un rôle dans les résultats, surtout sur des datasets avec des variations imprévues.

---

### Pourquoi envisager d'autres méthodes d'imputation ?
- **Imputation par la médiane** :
  - Pour des colonnes comme `GarageYrBlt`, où les valeurs suivent une distribution non uniforme, la médiane peut mieux capturer les tendances.
- **Imputation par une valeur spécifique** :
  - Par exemple, pour indiquer qu’il n’y a pas de garage, une valeur spécifique (comme -1 ou "NA") pourrait être plus pertinente que 0 ou la moyenne.
- **Imputation par la modalité (valeur la plus fréquente)** :
  - Pour des colonnes catégoriques ou avec des valeurs récurrentes, la modalité est souvent plus appropriée que la moyenne.

---

### Recommandations :
- Explorez différentes stratégies d'imputation pour chaque colonne (par exemple, médiane pour `GarageYrBlt`, 0 pour des valeurs numériques simples, etc.).
- Comparez les performances avec la fonction `score_dataset()` pour identifier l'approche qui donne les meilleurs résultats.

Cela met en lumière l’importance de bien comprendre la nature des données avant d’appliquer des techniques de manipulation des valeurs manquantes.

## Étape 4A : Prétraitement des données en supprimant les colonnes avec des valeurs manquantes
Dans cette méthode, nous supprimons toutes les colonnes qui contiennent des valeurs manquantes dans les données d'entraînement et appliquons la même modification aux données de validation et de test pour garantir la cohérence.

In [49]:
# Appliquer la suppression des colonnes sur les données d'entraînement, de validation et de test
final_X_train = X_train.drop(cols_with_missing, axis=1)
final_X_valid = X_valid.drop(cols_with_missing, axis=1)
final_X_test = X_test.drop(cols_with_missing, axis=1)

# Vérifier les dimensions
print(f"Dimensions des données d'entraînement : {final_X_train.shape}")
print(f"Dimensions des données de validation : {final_X_valid.shape}")
print(f"Dimensions des données de test : {final_X_test.shape}")


Dimensions des données d'entraînement : (1168, 33)
Dimensions des données de validation : (292, 33)
Dimensions des données de test : (1459, 33)


**Entraînez un modèle Random Forest sur les données nettoyées**

In [52]:
# Définir et entraîner le modèle Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Prédictions sur les données de validation
preds_valid = model.predict(final_X_valid)

# Calcul de la MAE
print("MAE (Suppression des colonnes avec des valeurs manquantes) :")
print(mean_absolute_error(y_valid, preds_valid))


MAE (Suppression des colonnes avec des valeurs manquantes) :
17837.82570776256


**Utilisez le modèle entraîné pour générer les prédictions sur les données de test :**

In [55]:
# Générer les prédictions sur les données de test
preds_test = model.predict(final_X_test)

# Sauvegarder les prédictions dans un fichier CSV
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('submission2.csv', index=False)

## Étape 4B : Prétraitement des données "Imputation avec la médiane"

In [60]:
# Nous utiliserons SimpleImputer avec la stratégie "median".

from sklearn.impute import SimpleImputer

# Imputation avec la médiane
imputer = SimpleImputer(strategy='median')

# Appliquer l'imputation sur les ensembles d'entraînement et de validation
final_X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
final_X_valid = pd.DataFrame(imputer.transform(X_valid), columns=X_valid.columns, index=X_valid.index)

# Vérification des dimensions
print(f"final_X_train dimensions: {final_X_train.shape}")
print(f"final_X_valid dimensions: {final_X_valid.shape}")


final_X_train dimensions: (1168, 36)
final_X_valid dimensions: (292, 36)


In [62]:
# Appliquer l'imputation sur les données de test
final_X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)

# Vérification des valeurs manquantes
print(f"Valeurs manquantes dans final_X_test :\n{final_X_test.isnull().sum().sum()}")


Valeurs manquantes dans final_X_test :
0


In [66]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Entraînement du modèle
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Évaluation sur les données de validation
preds_valid = model.predict(final_X_valid)
print("MAE (Imputation avec médiane):")
print(mean_absolute_error(y_valid, preds_valid))

# Prédictions sur l'ensemble de test
preds_test = model.predict(final_X_test)

# Sauvegarde des résultats
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('submission2b.csv', index=False)

print("Les prédictions ont été enregistrées dans le fichier 'submission.csv'.")


MAE (Imputation avec médiane):
17791.59899543379
Les prédictions ont été enregistrées dans le fichier 'submission.csv'.
